In [1]:
import pandas as pd
import json
import ast

In [2]:
# Read the dataset
df = pd.read_csv('laptop_data.csv')

In [3]:
!pip install -q google-generativeai


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
!pip install -q python-dotenv


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Load the API key from .env

In [20]:
import os
from dotenv import load_dotenv

# Load the environment variables from the .env file
load_dotenv()

# Get the API key from the environment
api_key = os.environ.get("GOOGLE_API_KEY")

if not api_key:
    raise ValueError("API key not found. Please create a .env file and add your GOOGLE_API_KEY.")

# Configure the genai library with the key
genai.configure(api_key=api_key)

print("API key configured successfully from .env file!")

API key configured successfully from .env file!


In [5]:
import pandas as pd

# Load the original data
df = pd.read_csv('laptop_data.csv')

# Add the pre-processed feature dictionaries for each laptop
laptop_features = [
    {'GPU intensity': 'low', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'low', 'Processing speed': 'medium'},
    {'GPU intensity': 'medium', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'medium', 'Processing speed': 'high'},
    {'GPU intensity': 'low', 'Display quality': 'medium', 'Portability': 'high', 'Multitasking': 'medium', 'Processing speed': 'high'},
    {'GPU intensity': 'low', 'Display quality': 'low', 'Portability': 'medium', 'Multitasking': 'low', 'Processing speed': 'low'},
    {'GPU intensity': 'high', 'Display quality': 'high', 'Portability': 'medium', 'Multitasking': 'high', 'Processing speed': 'high'},
    {'GPU intensity': 'medium', 'Display quality': 'medium', 'Portability': 'low', 'Multitasking': 'medium', 'Processing speed': 'high'},
    {'GPU intensity': 'medium', 'Display quality': 'medium', 'Portability': 'high', 'Multitasking': 'low', 'Processing speed': 'medium'},
    {'GPU intensity': 'medium', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'medium', 'Processing speed': 'high'},
    {'GPU intensity': 'low', 'Display quality': 'low', 'Portability': 'medium', 'Multitasking': 'low', 'Processing speed': 'medium'},
    {'GPU intensity': 'high', 'Display quality': 'medium', 'Portability': 'low', 'Multitasking': 'medium', 'Processing speed': 'high'},
    {'GPU intensity': 'high', 'Display quality': 'high', 'Portability': 'medium', 'Multitasking': 'high', 'Processing speed': 'high'},
    {'GPU intensity': 'low', 'Display quality': 'medium', 'Portability': 'high', 'Multitasking': 'medium', 'Processing speed': 'high'},
    {'GPU intensity': 'medium', 'Display quality': 'medium', 'Portability': 'high', 'Multitasking': 'low', 'Processing speed': 'medium'},
    {'GPU intensity': 'medium', 'Display quality': 'high', 'Portability': 'high', 'Multitasking': 'medium', 'Processing speed': 'high'},
    {'GPU intensity': 'medium', 'Display quality': 'medium', 'Portability': 'high', 'Multitasking': 'medium', 'Processing speed': 'medium'},
    {'GPU intensity': 'medium', 'Display quality': 'medium', 'Portability': 'high', 'Multitasking': 'medium', 'Processing speed': 'high'},
    {'GPU intensity': 'high', 'Display quality': 'high', 'Portability': 'medium', 'Multitasking': 'high', 'Processing speed': 'high'},
    {'GPU intensity': 'medium', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'medium', 'Processing speed': 'high'},
    {'GPU intensity': 'high', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'medium', 'Processing speed': 'high'},
    {'GPU intensity': 'medium', 'Display quality': 'high', 'Portability': 'medium', 'Multitasking': 'high', 'Processing speed': 'high'}
]

df['laptop_feature'] = [str(feature) for feature in laptop_features]

# Save the new CSV file
df.to_csv('updated_laptop.csv', index=False)

print("'updated_laptop.csv' has been created successfully!")

'updated_laptop.csv' has been created successfully!


In [6]:
def find_laptops(budget: int, portability: str = None, gpu_intensity: str = None, display_quality: str = None, multitasking: str = None, processing_speed: str = None):
    """
    Finds and compares laptops from a database based on user specifications.

    Args:
        budget (int): The user's maximum budget in INR.
        portability (str, optional): How important portability is. Options: 'low', 'medium', 'high'.
        gpu_intensity (str, optional): How important GPU intensity is. Options: 'low', 'medium', 'high'.
        display_quality (str, optional): How important display quality is. Options: 'low', 'medium', 'high'.
        multitasking (str, optional): How important multitasking is. Options: 'low', 'medium', 'high'.
        processing_speed (str, optional): How important processing speed is. Options: 'low', 'medium', 'high'.

    Returns:
        A list of dictionaries, where each dictionary represents a recommended laptop.
    """
    
    # Load the pre-processed laptop data
    laptop_df = pd.read_csv('updated_laptop.csv')

    # Create a dictionary of user requirements from the arguments
    user_requirements = {
        'Budget': str(budget), # Keep as string to match scoring logic
        'Portability': portability,
        'GPU intensity': gpu_intensity,
        'Display quality': display_quality,
        'Multitasking': multitasking,
        'Processing speed': processing_speed
    }
    
    # Filter out None values
    user_requirements = {k: v for k, v in user_requirements.items() if v is not None}

    # Filter by budget
    filtered_laptops = laptop_df.copy()
    filtered_laptops['Price'] = filtered_laptops['Price'].str.replace(',', '').astype(int)
    filtered_laptops = filtered_laptops[filtered_laptops['Price'] <= budget].copy()

    # Define scoring map
    mappings = {'low': 0, 'medium': 1, 'high': 2}
    filtered_laptops['Score'] = 0

    # Calculate score for each laptop
    for index, row in filtered_laptops.iterrows():
        laptop_values = ast.literal_eval(row['laptop_feature'])
        score = 0
        for key, user_value in user_requirements.items():
            if key == 'Budget':
                continue
            
            laptop_value = laptop_values.get(key, None)
            
            # Map string values to numerical scores for comparison
            laptop_mapping = mappings.get(laptop_value, -1)
            user_mapping = mappings.get(user_value, -1)
            
            if laptop_mapping >= user_mapping:
                score += 1
        filtered_laptops.loc[index, 'Score'] = score
    
    # Sort by score and select top 3
    top_laptops = filtered_laptops.sort_values('Score', ascending=False).head(3)
    
    # Return the results as a dictionary (the library will handle JSON conversion)
    output_columns = ['Brand', 'Model Name', 'Price', 'Description', 'Score']
    return top_laptops[output_columns].to_dict(orient='records')

In [8]:
from google.generativeai import protos

# Define the function declaration with the correct types
find_laptops_declaration = FunctionDeclaration(
    name="find_laptops",
    description="Finds and compares laptops from a database based on user specifications.",
    parameters=protos.Schema(
        type=protos.Type.OBJECT,
        properties={
            'budget': protos.Schema(type=protos.Type.INTEGER, description="The user's maximum budget in Indian Rupees (INR)."),
            'portability': protos.Schema(type=protos.Type.STRING, description="The user's preference for portability.", enum=['low', 'medium', 'high']),
            'gpu_intensity': protos.Schema(type=protos.Type.STRING, description="The user's requirement for GPU performance.", enum=['low', 'medium', 'high']),
            'display_quality': protos.Schema(type=protos.Type.STRING, description="The user's need for display quality.", enum=['low', 'medium', 'high']),
            'multitasking': protos.Schema(type=protos.Type.STRING, description="The user's requirement for multitasking capability.", enum=['low', 'medium', 'high']),
            'processing_speed': protos.Schema(type=protos.Type.STRING, description="The user's need for processing speed.", enum=['low', 'medium', 'high'])
        },
        required=['budget']
    )
)

# Define the tool
laptop_tool = protos.Tool(
    function_declarations=[find_laptops_declaration]
)

print("Tool definition successful!")

Tool definition successful!


In [13]:
import google.generativeai as genai
import pandas as pd
import ast

# --- Step 1: Initialize the Model with our Tool ---
print("Initializing the model...")
model = genai.GenerativeModel(
    model_name='models/gemini-pro-latest',
    tools=[laptop_tool]
)

# Start a chat session
chat = model.start_chat()
print("Model initialized.")
print("-" * 30)

# --- Step 2: Send a User Prompt ---
print("Sending user prompt to the model...")
prompt = "I need a powerful laptop for gaming, and my budget is 80000 INR."
response = chat.send_message(prompt)
print(f"User Prompt: '{prompt}'")
print("-" * 30)

# --- Step 3: Inspect the Model's Response (The Tool Call Request) ---
print("Model has responded. Let's see if it wants to call our tool...")
function_call = response.candidates[0].content.parts[0].function_call
print("Yes! The model wants to call the following function:")
print(function_call)
print("-" * 30)

# --- Step 4: Execute the Python Function ---
print("Executing our Python 'find_laptops' function with the arguments from the model...")
function_name = function_call.name
args = {key: value for key, value in function_call.args.items()}

results = None
if function_name == 'find_laptops':
    results = find_laptops(**args)
    print("Function executed successfully. Laptops found:")
    print(json.dumps(results, indent=4))
print("-" * 30)

# --- Step 5: Send the Results Back to the Model ---
if results:
    print("Sending the function results back to the model...")
    # CORRECTED: Pass the Part object directly, without the 'part=' keyword.
    final_response = chat.send_message(
        genai.protos.Part(
            function_response=genai.protos.FunctionResponse(
                name='find_laptops',
                response={
                    "laptops": results
                }
            )
        )
    )
    print("Results sent.")
    print("-" * 30)

    # --- Step 6: Get the Final, Natural Language Answer ---
    print("Final answer from the AI:")
    print(final_response.text)
else:
    print("Could not execute the function call.")

Initializing the model...
Model initialized.
------------------------------
Sending user prompt to the model...
User Prompt: 'I need a powerful laptop for gaming, and my budget is 80000 INR.'
------------------------------
Model has responded. Let's see if it wants to call our tool...
Yes! The model wants to call the following function:
name: "find_laptops"
args {
  fields {
    key: "processing_speed"
    value {
      string_value: "high"
    }
  }
  fields {
    key: "portability"
    value {
      string_value: "low"
    }
  }
  fields {
    key: "multitasking"
    value {
      string_value: "high"
    }
  }
  fields {
    key: "gpu_intensity"
    value {
      string_value: "high"
    }
  }
  fields {
    key: "display_quality"
    value {
      string_value: "high"
    }
  }
  fields {
    key: "budget"
    value {
      number_value: 80000
    }
  }
}

------------------------------
Executing our Python 'find_laptops' function with the arguments from the model...
Function execu

        All at once with Proper comments 

In [22]:
# --- Imports and Setup ---
import google.generativeai as genai
import pandas as pd
import ast
import json
from google.generativeai import protos

# --- API Key Configuration ---
# Ensure your API key is configured. Replace "YOUR_API_KEY" with your actual key.
# genai.configure(api_key="YOUR_API_KEY")


# --- Tool Definition ---

def find_laptops(budget: int, portability: str = None, gpu_intensity: str = None, display_quality: str = None, multitasking: str = None, processing_speed: str = None):
    """
    Finds and compares laptops from a CSV database based on user specifications.

    This function acts as the primary "tool" for the AI. It filters laptops by budget,
    then calculates a matching score based on other user preferences.

    Args:
        budget (int): The user's maximum budget in Indian Rupees (INR).
        portability (str, optional): The user's preference for portability ('low', 'medium', 'high').
        gpu_intensity (str, optional): The user's requirement for GPU performance ('low', 'medium', 'high').
        display_quality (str, optional): The user's need for display quality ('low', 'medium', 'high').
        multitasking (str, optional): The user's requirement for multitasking capability ('low', 'medium', 'high').
        processing_speed (str, optional): The user's need for processing speed ('low', 'medium', 'high').

    Returns:
        list: A list of up to 3 dictionaries, where each dictionary represents a recommended laptop.
    """
    # Load the pre-processed laptop data from the CSV file.
    laptop_df = pd.read_csv('updated_laptop.csv')

    # Create a dictionary of user requirements from the function arguments.
    user_requirements = {
        'Budget': str(budget),
        'Portability': portability,
        'GPU intensity': gpu_intensity,
        'Display quality': display_quality,
        'Multitasking': multitasking,
        'Processing speed': processing_speed
    }
    
    # Filter out any preferences the user didn't specify.
    user_requirements = {k: v for k, v in user_requirements.items() if v is not None}

    # First, filter the DataFrame to only include laptops within the user's budget.
    filtered_laptops = laptop_df.copy()
    filtered_laptops['Price'] = filtered_laptops['Price'].str.replace(',', '').astype(int)
    filtered_laptops = filtered_laptops[filtered_laptops['Price'] <= budget].copy()

    # Define a numerical mapping for text-based preferences to make them comparable.
    mappings = {'low': 0, 'medium': 1, 'high': 2}
    filtered_laptops['Score'] = 0

    # Iterate through each laptop to calculate its matching score.
    for index, row in filtered_laptops.iterrows():
        laptop_values = ast.literal_eval(row['laptop_feature'])
        score = 0
        for key, user_value in user_requirements.items():
            if key == 'Budget': continue  # Skip budget as it was already used for filtering.
            
            laptop_value = laptop_values.get(key, None)
            
            # Convert text preferences to numerical scores.
            laptop_mapping = mappings.get(laptop_value, -1)
            user_mapping = mappings.get(user_value, -1)
            
            # A laptop gets a point if its feature meets or exceeds the user's requirement.
            if laptop_mapping >= user_mapping:
                score += 1
        filtered_laptops.loc[index, 'Score'] = score
    
    # Sort laptops by their score and select the top 3.
    top_laptops = filtered_laptops.sort_values('Score', ascending=False).head(3)
    
    # Return only the essential columns for the AI to use.
    output_columns = ['Brand', 'Model Name', 'Price', 'Description', 'Score']
    return top_laptops[output_columns].to_dict(orient='records')


# --- Tool Declaration ---

# This schema describes the 'find_laptops' function to the AI.
find_laptops_declaration = protos.FunctionDeclaration(
    name="find_laptops",
    description="Finds and compares laptops from a database based on user specifications.",
    parameters=protos.Schema(
        type=protos.Type.OBJECT,
        properties={
            'budget': protos.Schema(type=protos.Type.INTEGER, description="The user's maximum budget in INR."),
            'portability': protos.Schema(type=protos.Type.STRING, description="Portability preference.", enum=['low', 'medium', 'high']),
            'gpu_intensity': protos.Schema(type=protos.Type.STRING, description="GPU performance requirement.", enum=['low', 'medium', 'high']),
            'display_quality': protos.Schema(type=protos.Type.STRING, description="Display quality preference.", enum=['low', 'medium', 'high']),
            'multitasking': protos.Schema(type=protos.Type.STRING, description="Multitasking capability requirement.", enum=['low', 'medium', 'high']),
            'processing_speed': protos.Schema(type=protos.Type.STRING, description="Processing speed preference.", enum=['low', 'medium', 'high'])
        },
        required=['budget'] # The AI must get a budget from the user before calling the tool.
    )
)
# We create a 'Tool' object that the model can use.
laptop_tool = protos.Tool(function_declarations=[find_laptops_declaration])


# --- Main Chatbot Application ---

def run_chatbot():
    """
    Initializes and runs the main interactive loop for the ShopAssist 2.0 chatbot.
    """
    print("Initializing ShopAssist 2.0...")
    # Initialize the generative model and make it aware of our tool.
    model = genai.GenerativeModel(model_name='models/gemini-pro-latest', tools=[laptop_tool])
    chat = model.start_chat()
    print("ShopAssist 2.0 is ready! Ask me to find a laptop for you. Type 'quit' or 'exit' to end.")
    print("-" * 30)

    # Start a continuous loop to wait for user input.
    while True:
        user_input = input("You: ")

        # Define an exit condition.
        if user_input.lower() in ["quit", "exit"]:
            print("Goodbye!")
            break

        # Send the user's message to the model.
        response = chat.send_message(user_input)

        try:
            # Check if the model's response contains a request to call our function.
            function_call = response.candidates[0].content.parts[0].function_call
            print("AI requested a tool call...")
            
            # --- If it's a tool call, execute the function ---
            function_name = function_call.name
            args = {key: value for key, value in function_call.args.items()}
            
            if function_name == 'find_laptops':
                # Execute the actual Python function with the arguments extracted by the AI.
                results = find_laptops(**args)

                if results:
                    # Send the function's results back to the model.
                    final_response = chat.send_message(
                        protos.Part(function_response=protos.FunctionResponse(name='find_laptops', response={"laptops": results}))
                    )
                    # Print the AI's final, natural-language summary.
                    print(f"ShopAssist: {final_response.text}")
                else:
                    print("ShopAssist: I couldn't find any laptops that match your criteria.")
            
        except (AttributeError, IndexError):
            # --- If it's a regular text response, just print it ---
            print(f"ShopAssist: {response.text}")

# To start the chatbot, call this function in your script or notebook.
# run_chatbot()

In [21]:
run_chatbot()

Initializing ShopAssist 2.0...
ShopAssist 2.0 is ready! Ask me to find a laptop for you. Type 'quit' or 'exit' to end.
------------------------------
AI requested a tool call...
ShopAssist: Of course, I can help you with that. Could you please provide some more details about your requirements? For instance, what are your preferences for processing speed, multitasking, portability, and display quality?
AI requested a tool call...
ShopAssist: I can help with that. What about other specifications like processing speed, multitasking, and display quality?
AI requested a tool call...
ShopAssist: Here are a few laptops that have a good balance of features for your needs:

* **MSI GL65**: Priced at ₹55,000, this is a high-performance gaming laptop with an Intel Core i7 processor, 16GB of RAM, and a 15.6" IPS display. It features an NVIDIA GTX graphics card and an RGB keyboard.

* **MSI Prestige 14**: Priced at ₹70,000, this is a compact and stylish laptop designed for professionals and content